# Check funding rate entities

* Query positions to ensure `netFunding` is recorded properly
* Check `fundingRates` value on the entities for historical funding rate chart


In [30]:
import plotly.io as pio
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()

pio.renderers.default = 'notebook'

In [31]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [32]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [55]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      timestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
    }  
}
""")

funding_query = gql("""
query funding(
    $last_id: ID!
  ) {
    fundingRateUpdates(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      timestamp
      market
      marketKey
      asset
      funding
      fundingRate
      sequenceLength
    }  
}
""")


### Run a query

In [56]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

position_bytes_cols = [
    'asset'
]

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, decimal_cols=position_cols, bytes_cols=position_bytes_cols)


In [57]:
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,timestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x0,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,77.199529739466510105,0,1.336,1.336,1.240604731598298524,NaN,0,...,1677518771,NaN,0,0,False,True,1609.083283518416578541,7195.93651595228688712,627578.390529561362089078,0
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,1675952960,1675952960,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,MATIC,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,1676274397,1676274397,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x100,0x2c23f12195077bee4ac4f7d598af30904a5fbf91,MATIC,98,0,1.271783672974139673,1.271783672974139673,1.268341831466371221,1.268341831466371221,6.767650966258712392,...,1677309680,1677309680,8.993264259656970653,-0.010044854998744103,False,False,97.764341851602997636,0,0,4994.617789850222396638
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x101,0xd8e80c06f9f3a41c85cad28117e01ebd50fd535e,MATIC,777.321876625415779658,0,1.257577965021184482,1.257577965021184482,1.205252195554925268,1.205252195554925268,210.365882266396886773,...,1677358123,1677358123,13.943417100998227914,-1.755761096539670812,False,False,973.988580694274767704,0,0,9901.343924544948537964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6578,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,XAG,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,1676542759,1676542759,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538
6579,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,XAG,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,1676587925,1676587925,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318
6580,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,XAG,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,1676883214,1676883214,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407
6581,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,XAG,183.427032240973901673,0,21.728166360389986619,21.728166360389986619,21.816099329108998862,21.816099329108998862,14.574525202046228121,...,1677033107,1677033107,5.081863514639043735,-0.794828372462505745,False,False,194.124865555918580315,0,0,7217.281605994306985757


In [58]:
df_position.loc[df_position['netFunding'].abs() > 0, :]

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,timestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,1675952960,1675952960,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,MATIC,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,1676274397,1676274397,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x100,0x2c23f12195077bee4ac4f7d598af30904a5fbf91,MATIC,98,0,1.271783672974139673,1.271783672974139673,1.268341831466371221,1.268341831466371221,6.767650966258712392,...,1677309680,1677309680,8.993264259656970653,-0.010044854998744103,False,False,97.764341851602997636,0,0,4994.617789850222396638
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x101,0xd8e80c06f9f3a41c85cad28117e01ebd50fd535e,MATIC,777.321876625415779658,0,1.257577965021184482,1.257577965021184482,1.205252195554925268,1.205252195554925268,210.365882266396886773,...,1677358123,1677358123,13.943417100998227914,-1.755761096539670812,False,False,973.988580694274767704,0,0,9901.343924544948537964
5,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x102,0x20bffd797ef3460e0283800ca5039b1c30dc764e,MATIC,2790.254775270753408915,6767.83,1.257684245661669336,1.257684245661669336,1.244894655096965818,NaN,-55.277651493320571662,...,1677500119,NaN,21.614728723355811239,33.544204515769318634,False,True,2662.348824858328934422,0,0,19328.142887317758470749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6578,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,XAG,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,1676542759,1676542759,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538
6579,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,XAG,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,1676587925,1676587925,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318
6580,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,XAG,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,1676883214,1676883214,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407
6581,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,XAG,183.427032240973901673,0,21.728166360389986619,21.728166360389986619,21.816099329108998862,21.816099329108998862,14.574525202046228121,...,1677033107,1677033107,5.081863514639043735,-0.794828372462505745,False,False,194.124865555918580315,0,0,7217.281605994306985757


In [59]:
df_position['pnl'].sum() + df_position['netFunding'].sum() - df_position['feesPaid'].sum()

Decimal('-223462.184019250510069233')

In [60]:
df_position['feesPaid'].sum()

Decimal('503601.334540086807084891')

In [61]:
df_position['pnlWithFeesPaid'].sum()



Decimal('-212053.184019250510069233')

## Historical funding rate

In [62]:
funding_params = {
    'last_id': ''
}

funding_decimal_cols = [
    'funding',
    'fundingRate'
]

funding_bytes_cols = [
    'marketKey',
    'asset'
]

df_funding = await run_recursive_query(funding_query, funding_params, 'fundingRateUpdates')
df_funding = df_funding.replace({None: np.nan})
df_funding = clean_df(df_funding, decimal_cols=funding_decimal_cols, bytes_cols=funding_bytes_cols)


In [63]:
df_funding['timestamp'] = df_funding['timestamp'].astype(np.int64)
df_funding['fundingRate'] = df_funding['fundingRate'].astype(np.float64)


In [64]:
df_funding

,id,timestamp,market,marketKey,asset,funding,fundingRate,sequenceLength
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1,1675952475,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,0,0.000000e+00,1
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-10,1675960087,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-1.9047554247E-8,1.699673e-07,10
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-100,1676082221,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.001322592712477598,1.551538e-03,100
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1000,1676748461,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.023724414029589254,5.880351e-04,1000
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1001,1676750878,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.023747721570361566,5.330023e-04,1001
...,...,...,...,...,...,...,...,...
43669,0xdcb8438c979fa030581314e5a5df42bbfed744a0-95,1677033107,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009786980809347932,1.269254e-04,95
43670,0xdcb8438c979fa030581314e5a5df42bbfed744a0-96,1677033152,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009788424095941762,1.269323e-04,96
43671,0xdcb8438c979fa030581314e5a5df42bbfed744a0-97,1677038477,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009959739826614255,1.277435e-04,97
43672,0xdcb8438c979fa030581314e5a5df42bbfed744a0-98,1677177995,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.014718547604870155,1.489971e-04,98


In [65]:
df_funding_sorted = df_funding.sort_values(['asset', 'timestamp'])


In [66]:
fig = px.line(
    df_funding_sorted,
    x='timestamp',
    y='fundingRate',
    color='asset'
)

fig.show()

## Look at a specific market

In [76]:
asset = 'MATIC'
last_price = df_position[(df_position['asset'] == asset)].sort_values('timestamp', ascending=False).iloc[0, :]['lastPrice']

df_position['sizeUsd'] = df_position.apply(lambda x: x['size'] * last_price, axis=1)
df_long = df_position.loc[(df_position['asset'] == asset) & (df_position['isOpen'] == True) & (df_position['size'] > 0), :].sort_values('size', ascending=False)
df_short = df_position.loc[(df_position['asset'] == asset) & (df_position['isOpen'] == True) & (df_position['size'] < 0), :].sort_values('size', ascending=True)

long_oi = df_long['size'].sum()
short_oi = abs(df_short['size'].sum())
total_oi = long_oi + short_oi

df_long['oiSide'] = df_long['size'].apply(lambda x: x / long_oi)
df_short['oiSide'] = df_short['size'].apply(lambda x: -1 * x / short_oi)
df_long['oiTotal'] = df_long['size'].apply(lambda x: x / total_oi)
df_short['oiTotal'] = df_short['size'].apply(lambda x: -1 * x / total_oi)

display_cols = [
    'account',
    'asset',
    'margin',
    'size',
    'sizeUsd',
    'oiSide',
    'oiTotal',
    'avgEntryPrice',
    'pnl',
    'totalVolume'
]


In [77]:
df_long[display_cols]

,account,asset,margin,size,sizeUsd,oiSide,oiTotal,avgEntryPrice,pnl,totalVolume
257,0xcd4ef8cd33a4ef7a59b368e1a72249b5eb5c4ece,MATIC,29174.491966372037968431,43384.0106,53828.3780689464023475855258,0.2352601495967351993501431957,0.1107300836182053149717739418,1.298954054172366384,-431.479073061071533621,105183.541701371888933154
38,0x9d99411fa69be2325d8542008d6f668f545a4a83,MATIC,1542.489823114608300414,33662.3161,41766.2602568878542036453873,0.1825419414188181995447197045,0.08591716222140741933342412117,1.240604731598298524,0,41761.628630217583137051
26,0xbcbeac56eef250e8a4859be46c6cbfd93aae5d2f,MATIC,7506.56014912652280935,26000,32259.300383644255818000,0.1409912040154977092667343920,0.06636044326601142286430573773,1.268980090083889909,0,32993.482342181137647
264,0x590ce964c5bc43b8903b013631c64f7bcf950b99,MATIC,9272.825016085143807651,18746.971977589312400419,23260.16155033903918084569908,0.1016599288755812805464423374,0.04784836039724313588896149210,1.266861003660569769,-194.921611705819726027,41929.985290132088009196
212,0x12a1e8f498501c2cce0967f8b9717b28492fc668,MATIC,2373.611413345426901007,10990.4772,13636.3502059382094799498596,0.05959848512049523155185665590,0.02805126687296123376652734247,1.323469915150422713,0,14545.565927346655413446
29,0x16d1663a00d4d1a216e0baa84b0abc69ba35c156,MATIC,3596.89582419142600754,7309.5213,9069.2324337440715201661509,0.03963762341784330907640009306,0.01865627205885969467192114614,1.272534625124232286,0,9301.618947333091040664
271,0xd68fd7b6c28718f8a42faa7cc342a789e3748331,MATIC,741.171071364678511695,7083.1263,8788.3345756534891626309159,0.03840994250887836277296000334,0.01807843849912090020733020393,1.279284704792123051,0,9061.335137700822819372
5,0x20bffd797ef3460e0283800ca5039b1c30dc764e,MATIC,2662.348824858328934422,6767.83,8397.13311213227322510519,0.03670017308739253314256473154,0.01727369995188500338475901158,1.257684245661669336,-55.277651493320571662,19328.142887317758470749
188,0xac9f5a0a1c0f6862f2485b2f1390d31bfc90f4ab,MATIC,1120.055723085511174723,6417.032208869692095402,7961.883446055677097890442364,0.03479788835902948719878245137,0.01637835007049481404507617490,1.225647614744712073,0,7865.02022054112911817
249,0xc1bd21ecd832ef3c3b1a393a0d66dd2b92c0a944,MATIC,471.330376354806755561,5265.3826,6532.9830357005303528479218,0.02855279355293276033511238563,0.01343896628081321989811375356,1.363104371393486373,0,7177.266039119200902109


In [78]:
df_short[display_cols]


,account,asset,margin,size,sizeUsd,oiSide,oiTotal,avgEntryPrice,pnl,totalVolume
25,0x4fa9bcd00ecfec65c8431d9fa1174e306575958f,MATIC,4686.997522760374376533,-89283.4278,-110777.7275646774698158128054,0.4305077127770171221932124370,0.2278803017352664256646350204,1.277542040186083308,-90.673585571982168962,243675.260296639499914315
12,0x4f6fff5cd87eaf8cf09467366fe783e7ecb1317b,MATIC,3965.628745891407994753,-51990.2794,-64506.5400074689250070344442,0.2506872419960120536658994110,0.1326960764041454768579539843,1.253657994073961549,0,65178.02938394880522361
149,0x653d99f0bdf3ea2587763afb9713fa948ba3b728,MATIC,7576.80140694786412447,-27496.047,-34115.509212915403432759671,0.1325807106207380997487790478,0.07017884103781090713899327642,1.47580389891819338,0,40578.77336743789438768
16,0x70b3d1ff337bcceeb3e2cf5b05efbffc50af87d7,MATIC,1130.878217080518873046,-13375.6497,-16595.7346730269682784136521,0.06449483964877068975345170691,0.03413900165626510417044323775,1.25610171689837756,0,16801.176552801268751426
10,0xc15ab8dd7daeea5f1aa3b56507cac8a877e21b15,MATIC,1215.312431714301826994,-11237.7605,-13943.1650618827793878236765,0.05418634441800528993138137661,0.02868241415758746770820353382,1.256734861782301143,-713.206425880960740434,129060.503848812270514808
31,0x6c6bb330c5bfa5d47367b0f76b99ff65e12cbc16,MATIC,492.015370449509650876,-4005.9816,-4970.3909140673780674077288,0.01931608159025918911225942725,0.01022456594967252559170108008,1.244936145506831818,0,4987.191292075290937202
34,0x9482c72cb018ee03d8c23395038b510ed4e6040c,MATIC,3707.563071790014541623,-3812.1359,-4729.8783500528477089454487,0.01838139448208053305080914319,0.009729808773625976727350028129,1.267645189683840509,0,4832.435736056078054233
8,0x48f64de392581170695a74139512bcbc3913c9b3,MATIC,248.123738076347471708,-3419.604263327672668528,-4242.847735533766296625015369,0.01648868156479687634196580568,0.008727940565721800313967326229,1.246016362675881059,0,4260.882865982482463315
235,0xd80c45b242b5069a15f5094333eb0f6ece070deb,MATIC,315.906601655049818159,-1675.4717,-2078.8286482536574419468981,0.008078805968397425245921946674,0.004276347872756065803302969373,1.335560982306793178,0,2237.694629479232687492
234,0x6868484ab76eebc6a4e7158536d8df24db1d36fe,MATIC,136.604816842354606912,-1049.4477,-1302.0941765855608417850661,0.005060237270662912784653253956,0.002678531329096006765333579560,1.33244085765035439,0,1398.32699344719181877
